In [50]:
import os
import pandas as pd
import numpy as np
import scipy
import statsmodels
import sklearn
import datetime
import calendar
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from pandasql import sqldf
%matplotlib inline
import math
import collections
from operator import itemgetter
from collections import OrderedDict
from matplotlib.pyplot import show
from pandas import DataFrame
import seaborn as sns

In [51]:
os.chdir("/Users/chanduboss/Desktop/SUITEAMERICA")
os.getcwd()

'/Users/chanduboss/Desktop/SUITEAMERICA'

In [52]:
df = pd.read_csv("OrdersData_Timeseries.csv")
df.head()

,Order#,MoveInDate,MoveOutDate,DailyRent,SuiteSizeCode,Status,ZoneCode,City,State,GuestFirstName,Master#,ComplexName,BookedDate,StatusChanged,Unnamed: 14
0,285536,2015-09-14,2015-10-28,254.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,Matthew Richard,283368,AVALON HAYES VALLEY,2015-08-14,NaN,NaN
1,289693,2015-10-16,2015-11-14,250.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,Orion Richardson,119226,AVALON MISSION BAY PHASE I,2015-09-29,NaN,NaN
2,354867,2018-02-25,2018-03-26,NaN,NaN,CANCEL,NaN,NaN,NaN,RQ5C71AF,NaN,NaN,2018-01-31,NaN,NaN
3,355682,2018-02-19,2018-03-02,197.0,1X1,TERMED,7.0,SAN MATEO,US-CA,Thomas Spanswick,335345,PARK PLACE AT SAN MATEO,2018-02-09,NaN,NaN
4,322147,2016-12-17,2017-02-28,148.0,1X1,TERMED,46.0,HILLSBORO,US-OR,Michael Ulrich,301739,206 APARTMENTS,2016-11-17,NaN,NaN


In [53]:
df.shape

(46850, 15)

In [54]:
df.describe(include='all')

,Order#,MoveInDate,MoveOutDate,DailyRent,SuiteSizeCode,Status,ZoneCode,City,State,GuestFirstName,Master#,ComplexName,BookedDate,StatusChanged,Unnamed: 14
count,46850.000000,46777,43974,41806.000000,35213,46850,34560.000000,35180,35180,45186,35213,35207,46804,1306,0.0
unique,NaN,1546,1524,NaN,38,6,NaN,409,46,34874,6324,1660,1304,38,NaN
top,NaN,2018-04-01,2018-06-30,NaN,1X1,TERMED,NaN,CUPERTINO,US-CA,TBD,259056,APPLE,2018-02-20,2018-05-04,NaN
freq,NaN,296,194,NaN,21828,29349,NaN,6376,24298,717,2657,2709,159,68,NaN
mean,323875.540406,NaN,NaN,286.298409,NaN,NaN,14.090943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,31611.280139,NaN,NaN,421.372716,NaN,NaN,15.692640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5012.000000,NaN,NaN,0.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,298285.250000,NaN,NaN,150.000000,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,334230.500000,NaN,NaN,192.000000,NaN,NaN,7.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,350816.750000,NaN,NaN,236.000000,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df.dtypes

Order#              int64
MoveInDate         object
MoveOutDate        object
DailyRent         float64
SuiteSizeCode      object
Status             object
ZoneCode          float64
City               object
State              object
GuestFirstName     object
Master#            object
ComplexName        object
BookedDate         object
StatusChanged      object
Unnamed: 14       float64
dtype: object

In [56]:
df['MoveInDate'] = pd.to_datetime(df['MoveInDate'],format='%Y-%m-%d %H:%M')
df["MoveOutDate"] = pd.to_datetime(df['MoveOutDate'],format='%Y-%m-%d %H:%M')
df['MoveInDate_week'] = df['MoveInDate'].dt.weekday_name
df["MoveOutDay_Week"] = df["MoveOutDate"].dt.weekday_name
df['Move_In_Month']= df.MoveInDate.dt.month
df["Move_Out_Month"] = df.MoveOutDate.dt.month
df['Move_In_year']=df.MoveInDate.dt.year
df['Move_Out_Year'] = df.MoveOutDate.dt.year

In [57]:
#df["stay"] = (df["MoveOutDate"] - df["MoveInDate"]).dt.days

In [58]:
del df['Unnamed: 14']
del df['ComplexName']
del df['GuestFirstName']

In [59]:
df.dtypes.count

<bound method Series.count of Order#                      int64
MoveInDate         datetime64[ns]
MoveOutDate        datetime64[ns]
DailyRent                 float64
SuiteSizeCode              object
Status                     object
ZoneCode                  float64
City                       object
State                      object
Master#                    object
BookedDate                 object
StatusChanged              object
MoveInDate_week            object
MoveOutDay_Week            object
Move_In_Month             float64
Move_Out_Month            float64
Move_In_year              float64
Move_Out_Year             float64
dtype: object>

In [60]:
df.shape

(46850, 18)

In [61]:
df.head(15)

,Order#,MoveInDate,MoveOutDate,DailyRent,SuiteSizeCode,Status,ZoneCode,City,State,Master#,BookedDate,StatusChanged,MoveInDate_week,MoveOutDay_Week,Move_In_Month,Move_Out_Month,Move_In_year,Move_Out_Year
0,285536,2015-09-14,2015-10-28,254.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,283368,2015-08-14,NaN,Monday,Wednesday,9.0,10.0,2015.0,2015.0
1,289693,2015-10-16,2015-11-14,250.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,119226,2015-09-29,NaN,Friday,Saturday,10.0,11.0,2015.0,2015.0
2,354867,2018-02-25,2018-03-26,NaN,NaN,CANCEL,NaN,NaN,NaN,NaN,2018-01-31,NaN,Sunday,Monday,2.0,3.0,2018.0,2018.0
3,355682,2018-02-19,2018-03-02,197.0,1X1,TERMED,7.0,SAN MATEO,US-CA,335345,2018-02-09,NaN,Monday,Friday,2.0,3.0,2018.0,2018.0
4,322147,2016-12-17,2017-02-28,148.0,1X1,TERMED,46.0,HILLSBORO,US-OR,301739,2016-11-17,NaN,Saturday,Tuesday,12.0,2.0,2016.0,2017.0
5,305710,2016-05-29,2016-06-27,219.0,1X1,TERMED,5.0,CUPERTINO,US-CA,238110,2016-04-26,NaN,Sunday,Monday,5.0,6.0,2016.0,2016.0
6,348736,2017-11-26,NaT,NaN,NaN,CANCEL,NaN,NaN,NaN,NaN,2017-11-07,NaN,Sunday,NaN,11.0,NaN,2017.0,NaN
7,362371,2018-05-01,2018-05-21,202.0,1X1,ACTIVE,5.0,SUNNYVALE,US-CA,278650,2018-04-30,2018-04-30,Tuesday,Monday,5.0,5.0,2018.0,2018.0
8,361210,2018-05-09,2018-06-07,0.0,NaN,CANCEL,NaN,NaN,NaN,NaN,2018-04-13,NaN,Wednesday,Thursday,5.0,6.0,2018.0,2018.0
9,326135,2017-03-21,2017-04-15,236.0,2X2,TERMED,42.0,WASHINGTON,US-DC,326751,2017-02-02,NaN,Tuesday,Saturday,3.0,4.0,2017.0,2017.0


In [62]:
df['sum']=df.isnull().sum(axis=1)


In [63]:
df.describe()

,Order#,DailyRent,ZoneCode,Move_In_Month,Move_Out_Month,Move_In_year,Move_Out_Year,sum
count,46850.000000,41806.000000,34560.000000,46777.000000,43974.000000,46777.000000,43974.000000,46850.000000
mean,323875.540406,286.298409,14.090943,6.030977,6.423273,2016.597409,2016.746691,2.589840
std,31611.280139,421.372716,15.692640,3.374918,3.286883,1.281791,3.146523,2.725725
min,5012.000000,0.000000,1.000000,1.000000,1.000000,1994.000000,2015.000000,0.000000
25%,298285.250000,150.000000,5.000000,3.000000,4.000000,2016.000000,2016.000000,1.000000
50%,334230.500000,192.000000,7.000000,6.000000,6.000000,2017.000000,2017.000000,1.000000
75%,350816.750000,236.000000,13.000000,9.000000,9.000000,2018.000000,2018.000000,6.000000
max,363867.000000,3000.000000,77.000000,12.000000,12.000000,2106.000000,2099.000000,16.000000


In [64]:
df.isnull().sum()

Order#                 0
MoveInDate            73
MoveOutDate         2876
DailyRent           5044
SuiteSizeCode      11637
Status                 0
ZoneCode           12290
City               11670
State              11670
Master#            11637
BookedDate            46
StatusChanged      45544
MoveInDate_week       73
MoveOutDay_Week     2876
Move_In_Month         73
Move_Out_Month      2876
Move_In_year          73
Move_Out_Year       2876
sum                    0
dtype: int64

In [65]:
d = ["CANCEL","QUOTE","VOID"]
new_data = df[~df["Status"].isin(d)]

In [66]:
new_data.shape

(31171, 19)

In [67]:
new_data.head()

,Order#,MoveInDate,MoveOutDate,DailyRent,SuiteSizeCode,Status,ZoneCode,City,State,Master#,BookedDate,StatusChanged,MoveInDate_week,MoveOutDay_Week,Move_In_Month,Move_Out_Month,Move_In_year,Move_Out_Year,sum
0,285536,2015-09-14,2015-10-28,254.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,283368,2015-08-14,NaN,Monday,Wednesday,9.0,10.0,2015.0,2015.0,1
1,289693,2015-10-16,2015-11-14,250.0,1X1,TERMED,8.0,SAN FRANCISCO,US-CA,119226,2015-09-29,NaN,Friday,Saturday,10.0,11.0,2015.0,2015.0,1
3,355682,2018-02-19,2018-03-02,197.0,1X1,TERMED,7.0,SAN MATEO,US-CA,335345,2018-02-09,NaN,Monday,Friday,2.0,3.0,2018.0,2018.0,1
4,322147,2016-12-17,2017-02-28,148.0,1X1,TERMED,46.0,HILLSBORO,US-OR,301739,2016-11-17,NaN,Saturday,Tuesday,12.0,2.0,2016.0,2017.0,1
5,305710,2016-05-29,2016-06-27,219.0,1X1,TERMED,5.0,CUPERTINO,US-CA,238110,2016-04-26,NaN,Sunday,Monday,5.0,6.0,2016.0,2016.0,1


In [68]:

df_MI = pd.DataFrame(data= new_data.MoveInDate, index=None)
df_MI = df_MI.rename(columns={'MoveInDate': 'Date'})


In [69]:
df_MI['Date'] = pd.to_datetime(df_MI['Date'],format= '%d-%m-%Y')

In [70]:
df_MI['Date'] = pd.to_datetime(df_MI['Date'])


In [71]:
df_MI.head()

,Date
0,2015-09-14
1,2015-10-16
3,2018-02-19
4,2016-12-17
5,2016-05-29


In [72]:
df_MI.shape

(31171, 1)

In [78]:
df_MI['year']= df_MI['Date'].dt.year
df_MI
df_MI['Month']= df_MI['Date'].dt.month
df_MI['Weak']= df_MI['Date'].dt.weekday_name
#df_MI['Count']= 1

In [79]:
df_MI.head(20)

,Date,year,Month,Weak,Count
0,2015-09-14,2015,9,Monday,1970-01-01 00:00:00.000000028
1,2015-10-16,2015,10,Friday,1970-01-01 00:00:00.000000031
3,2018-02-19,2018,2,Monday,1970-01-01 00:00:00.000000012
4,2016-12-17,2016,12,Saturday,1970-01-01 00:00:00.000000014
5,2016-05-29,2016,5,Sunday,1970-01-01 00:00:00.000000035
7,2018-05-01,2018,5,Tuesday,1970-01-01 00:00:00.000000022
9,2017-03-21,2017,3,Tuesday,1970-01-01 00:00:00.000000018
10,2017-11-30,2017,11,Thursday,1970-01-01 00:00:00.000000015
12,2015-05-06,2015,5,Wednesday,1970-01-01 00:00:00.000000025
13,2016-07-22,2016,7,Friday,1970-01-01 00:00:00.000000019


In [80]:
df_MI = df_MI[df_MI["year"] > 2013]
df_MI = df_MI[df_MI["year"] < 2019]


In [81]:
df_MI.head()

,Date,year,Month,Weak,Count
0,2015-09-14,2015,9,Monday,1970-01-01 00:00:00.000000028
1,2015-10-16,2015,10,Friday,1970-01-01 00:00:00.000000031
3,2018-02-19,2018,2,Monday,1970-01-01 00:00:00.000000012
4,2016-12-17,2016,12,Saturday,1970-01-01 00:00:00.000000014
5,2016-05-29,2016,5,Sunday,1970-01-01 00:00:00.000000035


In [87]:
df_MI.Date.value_counts()


2018-01-21    173
2018-05-13    126
2015-05-23    111
2015-05-30     92
2015-06-13     90
2017-07-09     85
2015-05-16     76
2016-07-01     75
2016-03-01     73
2015-08-01     73
2015-07-01     73
2017-06-03     72
2017-01-15     70
2015-06-01     68
2016-06-04     68
2016-04-10     68
2016-05-15     68
2016-01-03     65
2016-05-01     62
2016-04-01     62
2017-06-18     60
2016-06-05     57
2016-01-10     57
2015-03-01     56
2015-05-31     56
2016-05-16     56
2017-10-01     55
2016-05-30     53
2015-08-15     53
2015-01-03     52
             ... 
2014-08-04      1
2018-07-16      1
2014-07-15      1
2014-07-24      1
2014-06-01      1
2014-08-03      1
2014-09-03      1
2014-08-08      1
2014-07-29      1
2014-03-26      1
2014-10-27      1
2014-08-19      1
2014-10-07      1
2018-10-01      1
2018-06-28      1
2014-08-26      1
2014-02-18      1
2014-10-21      1
2014-07-30      1
2014-06-30      1
2014-03-17      1
2014-03-29      1
2018-11-01      1
2014-06-21      1
2014-10-02

In [90]:
df_MI['Count'] = df_MI.groupby('Date')['Date'].transform('count')
df_MI


,Date,year,Month,Weak,Count
0,2015-09-14,2015,9,Monday,1970-01-01 00:00:00.000000028
1,2015-10-16,2015,10,Friday,1970-01-01 00:00:00.000000031
3,2018-02-19,2018,2,Monday,1970-01-01 00:00:00.000000012
4,2016-12-17,2016,12,Saturday,1970-01-01 00:00:00.000000014
5,2016-05-29,2016,5,Sunday,1970-01-01 00:00:00.000000035
7,2018-05-01,2018,5,Tuesday,1970-01-01 00:00:00.000000022
9,2017-03-21,2017,3,Tuesday,1970-01-01 00:00:00.000000018
10,2017-11-30,2017,11,Thursday,1970-01-01 00:00:00.000000015
12,2015-05-06,2015,5,Wednesday,1970-01-01 00:00:00.000000025
13,2016-07-22,2016,7,Friday,1970-01-01 00:00:00.000000019


In [42]:
min(df_MI['Date'])

Timestamp('2014-01-09 00:00:00')

In [43]:
max(df_MI.Date)

Timestamp('2018-11-01 00:00:00')

In [44]:
df_MI.dtypes

Date     datetime64[ns]
year              int64
Month             int64
Weak             object
Count             int64
dtype: object

In [ ]:
date,df_

In [45]:
#df_MI.Date= pd.to_datetime(df_MI.Date, format="%Y%m%d").sort_values()

In [47]:
#Final_TS= pd.DataFrame(df_MI.Date.sort_values())
#Final_TS

In [39]:

#Final_TS.head()

,Date
12870,2014-01-09
27005,2014-01-11
40660,2014-01-23
22077,2014-02-03
35148,2014-02-12


In [ ]:
#df_MI.to_csv('sortfile.csv')

In [ ]:


df_final = pd.read_csv("Final_TS.csv")
df_final.head()
df_final.tail()
df_final= df_final.sort_values(by='Date')



In [ ]:
del df_final['Unnamed: 0']

In [ ]:
print(df_final)

In [ ]:
TS = pd.read_csv("FINAL_TS.csv", parse_dates=["Date"], index_col="Date")
TS.head(2)

In [ ]:
#parse_dates=["Date"], index_col="Date")

In [ ]:
del TS['Unnamed: 0']


In [ ]:
TS.head(40)

In [ ]:
#TS['year'].resample('M').mean()

In [ ]:
TS.sort_index(by='Count', ascending=[False])

In [ ]:
TS.index

In [ ]:
print(TS)

In [ ]:
TS.index

In [ ]:
TS.Month.resample('M').mean()

In [ ]:
%matplotlib inline
TS['Month'].plot()